
<img src = "https://raw.githubusercontent.com/DevoLearn/devolearn/master/images/banner_1.jpg">

## Hello and welcome to devolearn !


### Install from PyPI

In [ ]:
!pip install devolearn --upgrade

### Install from Github

In [1]:
!pip install git+https://github.com/DevoLearn/devolearn

  Cloning https://github.com/DevoLearn/devolearn to /tmp/pip-req-build-m69q56la
  Running command git clone --filter=blob:none --quiet https://github.com/DevoLearn/devolearn /tmp/pip-req-build-m69q56la
  Resolved https://github.com/DevoLearn/devolearn to commit e3c4a1c9ed5d56b0ece3be2d38c709f9c429e0de
  Preparing metadata (setup.py) ... done
  Using cached efficientnet_pytorch-0.6.3.tar.gz (16 kB)
  Preparing metadata (setup.py) ... done
  Using cached imgaug-0.2.5.tar.gz (562 kB)
  Preparing metadata (setup.py) ... done
  Using cached imutils-0.5.3.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 1.4 MB/s eta 0:00:001.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/DevoLearn/devolearn.git
## for sample data

In [ ]:
from devolearn import Generator, embryo_generator_model
from devolearn import lineage_population_model
from devolearn import cell_nucleus_segmentor
from devolearn import cell_membrane_segmentora
import torch

import matplotlib.pyplot as plt
import os 
import cv2

## Importing a Pre-Trained model

The first time you'll run the cell below, devolearn will automatically download the latest model weights from GitHub

In [ ]:
model = lineage_population_model('cuda:0' if torch.cuda.is_available() else 'cpu')

## Estimating the lineage-wise population of cells within the C. elegans embryo from a photo  

In [ ]:
pred = model.predict(image_path = "devolearn/devolearn/tests/sample_data/images/embryo_sample.png")
print(pred)

## Estimating the same populations from a video file



One can directly save the frame-wise video predictions into a CSV file with `save_csv = True`

In [ ]:
if os.path.isdir("sample_preds") is not True:
    os.mkdir("sample_preds")

results = model.predict_from_video(video_path = "devolearn/devolearn/tests/sample_data/videos/embryo_timelapse.mov", 
                                   save_csv = True, 
                                   csv_name = "sample_preds/video_preds.csv", 
                                   ignore_first_n_frames= 10, 
                                   ignore_last_n_frames= 10 
                                   )


In [ ]:
results.tail(5)

##  Generating a population plot directly from a video file 

In [ ]:
plt.rcParams['figure.figsize'] = 15, 7
                                    
plot = model.create_population_plot_from_video(video_path = "devolearn/devolearn/tests/sample_data/videos/embryo_timelapse.mov", 
                                               save_plot= True, 
                                               plot_name= "sample_preds/plot.jpg", 
                                               ignore_last_n_frames= 0 
                                               )
plot.grid()
plot.show()

## Using the pre-trained GAN to synthetic images of embryos 

In [ ]:
generator = embryo_generator_model()

In [ ]:
gen_image = generator.generate()  ## returns a 2d numpy array 
plt.imshow(gen_image)
plt.show()

## Bulk generation using the GAN 

All the generated images get saved into the folder `foldername`

In [ ]:
generator.generate_n_images(n = 20, 
                            foldername= "generated_images", 
                            image_size= (700,500)  ## (width,height)
                            )

fig, ax = plt.subplots(3,3, figsize = (12,9))

image_paths = os.listdir("generated_images")
for i in range(9):
    image = cv2.imread("generated_images/" + image_paths[i], 0)
    ax.flat[i].imshow(image)


## Segmenting the C. elegans embryo 

In [ ]:
segmentor = cell_membrane_segmentor()

### Segmenting an image 

In [ ]:
seg_pred = segmentor.predict(image_path = "devolearn/devolearn/tests/sample_data/images/seg_sample.jpg", centroid_mode =False )
plt.imshow(seg_pred)
plt.show()

### Segmenting and extracting the centroids 

In [ ]:
seg_pred, centroids = segmentor.predict(image_path = "devolearn/devolearn/tests/sample_data/images/seg_sample.jpg", centroid_mode = True)
plt.imshow(seg_pred)
plt.show()

# Frame by frame segmentation from video files

All the segmented frames get saved into the folder `save_folder`

In [ ]:
filenames = segmentor.predict_from_video(video_path = "devolearn/devolearn/tests/sample_data/videos/seg_sample.mov", 
                                         centroid_mode = False, 
                                         save_folder = "segmented_images"
                                         )

In [ ]:
fig, ax = plt.subplots(3,3, figsize = (12,9))

segmented_image_paths = os.listdir("segmented_images")
for i in range(9):
    image = cv2.imread("segmented_images/" + segmented_image_paths[i], 0)
    ax.flat[i].imshow(image)

### Segmenting from a video and saving the centroids into a CSV file 



In [ ]:
df = segmentor.predict_from_video(video_path = "devolearn/devolearn/tests/sample_data/videos/seg_sample.mov", 
                                  centroid_mode = True, 
                                  save_folder = "centroid")
df.to_csv("centroids.csv")

In [ ]:
fig, ax = plt.subplots(3,3, figsize = (12,9))

segmented_image_paths = os.listdir("centroid")
for i in range(9):
    image = cv2.imread("centroid/" + segmented_image_paths[i], 0)
    ax.flat[i].imshow(image)

## Run tests (optional)

In [ ]:
import devolearn
os.path.dirname(devolearn.__file__) + "/" + "tests" + "/" + "test.py"
!python /content/devolearn/devolearn/tests/test.py